In [1]:
from __future__ import unicode_literals

In [2]:
import sys
sys.path.append('/Perfomer_local/keras_layer_normalization')
sys.path.append('/Perfomer_local/keras_performer')
sys.path.append('/Perfomer_local/keras_position_wise_feed_forward')
sys.path.append('/Perfomer_local/tensorflow_fast_attention')


In [3]:
import unittest
import numpy as np
from keras_performer import performer as tfr
import nltk
nltk.download('punkt')

from os import listdir
from os.path import isfile, isdir, join

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\W.R_Chen\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
import csv
def readCSV(file_name):
  errlist=[]
  LBlist=[]
  with open(file_name, newline='') as csvfile:

  # 讀取 CSV 檔內容，將每一列轉成一個 dictionary

    rows = csv.DictReader(csvfile)
    for row in rows: 
      RL=list(row.values())

      RL = list(map(int, RL))
      errs=RL[1:37]
      LB=RL[37:]
      errlist.append(errs)
      LBlist.append(LB)
  return errlist,LBlist

In [5]:
def parseSentence(x):	
	tokens=[]
	state="START"
	chrs=""
	for i in range(len(x)):
		#print(ord(x[i]))
		if (ord(x[i])>255):
			inp="U"
		else:
			inp="E"
	
		if state=="START":
			if inp=="E":
				state="ASCII"
				chrs=x[i]
			else:
				state="UNICODE"
				tokens.append(x[i])
			
		elif state=="ASCII":
			if inp=="E":
				chrs += x[i]
			else:#U
				state="UNICODE"
				tokens += nltk.word_tokenize(chrs)
				chrs=""
				tokens.append(x[i])
	
		elif state=="UNICODE":
			if inp=="E":
				state="ASCII"
				chrs=x[i]
			else:
				state="UNICODE"
				tokens.append(x[i])
	if len(chrs)>0:
		tokens += nltk.word_tokenize(chrs) 
	return tokens

In [6]:
def readcode(fname):
    with open(fname) as f:
        data = f.read()
        return data

In [7]:
class TestTranslate(unittest.TestCase):
 
    @staticmethod
    def _build_token_dict(token_list):
        token_dict = {
            '<PAD>': 0,
            '<START>': 1,
            '<END>': 2,
        }
        for tokens in token_list:
            for token in tokens:
                if token not in token_dict:
                    token_dict[token] = len(token_dict)
        return token_dict
 
    def test_translate(self):
        print("i am here: " )
        source_tokens = []
        target_errors=[]
        target_LB=[]
        #source_file=[]
        Input_Path = "D:\Code\Project-AI-JAVA-ANNOTATION-2021\Performer\Perfomer_local\Trianing\InputTxt"
        Location_Output_Path = "D:\Code\Project-AI-JAVA-ANNOTATION-2021\Performer\Perfomer_local\Trianing\InputCSV"
 
        InputFiles = sorted(listdir(Input_Path))
        OutputFiles = sorted(listdir(Location_Output_Path))
        #max_files =10
        for f in InputFiles:
          fullpath = join(Input_Path,f)
          #source_file=fullpath
          if isfile(fullpath):
            source_tokens.append(parseSentence(readcode(fullpath)))
            #if len(source_tokens)>max_files: break
        for f in OutputFiles:
          fullpath = join(Location_Output_Path,f)
          #source_file=fullpath
          if isfile(fullpath):
            err,lb = readCSV(fullpath)
            target_errors.append(err)
            target_LB.append(lb)
            #if len(source_tokens)>max_files: break
        dd=np.asarray(target_errors)
        print("AAAA: ", dd.shape)
        print("aaaa: " , type(target_errors[0][0]))
        target_errors=target_errors[0]  
        target_LB=target_LB[0]     
        print("XXXX: " , len(source_tokens))
        print("YYYY: " , type(target_errors[0][0]))
        print("ZZZZ: " , len(target_LB))

 
        source_tokens2 = []
        target_errors2= []
        target_LB2=[]

        THRESHOLD_FILE_LEN = 64000000
 
        for i in range(len(source_tokens)): 
          #print (i)
          src = source_tokens[i]
          err = target_errors[i]   
          lb = target_LB[i]                   
          if (len(src)<=THRESHOLD_FILE_LEN):
            source_tokens2.append(src)
            target_errors2.append(err)
            target_LB2.append(lb) 
        source_tokens = source_tokens2
        target_errors= target_errors2        
        target_LB= target_LB2        

        print("XXXX2: " , len(source_tokens))
                    
 
        # Generate dictionaries
        source_token_dict = self._build_token_dict(source_tokens)
        
 
        # Add special tokens
        encode_tokens = [['<START>'] + tokens + ['<END>'] for tokens in source_tokens]
        
        #output_tokens = [tokens + ['<END>', '<PAD>'] for tokens in target_tokens]
 
        # Padding
        sl = max(map(len, encode_tokens))
        
        source_max_len = sl
        
 
        encode_tokens = [tokens + ['<PAD>'] * (source_max_len - len(tokens)) for tokens in encode_tokens]
        
        
 
        encode_input = [list(map(lambda x: source_token_dict[x], tokens)) for tokens in encode_tokens]
       
        
        #print("encode_input:", encode_input)
        
        #print("len(source_token_dict):", len(source_token_dict))
        
        
        #decode_output_one_hot_encoded = to_categorical(decode_output)
        #print("one-hot-decode_output:", decode_output_one_hot_encoded)
        #decode_output =  list(decode_output_one_hot_encoded)
        token_num =len(source_token_dict)
        print(type(token_num))
         # Build & fit model
        model = tfr.get_model(
            max_input_len= (source_max_len),
            errNum=36,
            lbNum=192,
            token_num=len(source_token_dict),
            embed_dim=32,
            encoder_num=2,
            head_num=4,
            hidden_dim=128,
            dropout_rate=0.05
        )
        losses = {    
            "error_feed_forward_output1": "categorical_crossentropy",
            "error_feed_forward_output2": "mean_squared_error",            
        }
        lossWeights = {"error_feed_forward_output1": 1.0,"error_feed_forward_output2": 1.0}
        model.compile('adam', loss=losses, loss_weights=lossWeights)
        
        print("x.shape", np.asarray(encode_input).shape)  #x.shape (2,  9)
        
 
        x=[np.array(encode_input * 1)]
        y=[np.array(target_errors * 1),np.array(target_LB * 1)]
       
        print("x.shape", np.asarray(x).shape)  #x.shape (2, 2048, 9)
        
        model.fit(
            x=x,
            y=y,
            epochs=2,
            batch_size=32,
        )
        model.save("test_model.h5")
        saveDictionary(source_token_dict, 'source_token_dict.pickle')
        
        model, source_token_dict = load("test_model.h5")

        # Predict 
        out1, out2 = tfr.decode(
            model,
            encode_input,
            max_len=source_max_len
        )
        print("out1: ", out1)
        print("target_errors: ", target_errors)        
        print("out2: ", out2)
        print("target_LB: ", target_LB)   

In [8]:
def saveDictionary(dt, file):
        import pickle
        a_file = open(file, "wb")
        pickle.dump(dt, a_file)
        a_file.close()

In [9]:
def loadDictionary(file):
        import pickle
        a_file = open(file, "rb")
        dt = pickle.load(a_file)
        return dt

In [10]:
def load(model_name):
        import sys
        sys.path.append('/Perfomer_local/keras_position_wise_feed_forward')
        sys.path.append('/Perfomer_local/tensorflow_fast_attention')
        sys.path.append('/Perfomer_local/keras_performer')
        sys.path.append('/Perfomer_local/keras_pos_embed')

        from keras_performer import performer
        from tensorflow import keras
        from keras_embed_sim import EmbeddingRet, EmbeddingSim
        from keras_pos_embd import TrigPosEmbedding
        from tensorflow_fast_attention.fast_attention import  Attention, SelfAttention
        from keras_position_wise_feed_forward.feed_forward import FeedForward  

        co = performer.get_custom_objects()

        model = keras.models.load_model(model_name, custom_objects= co)
        source_token_dict = loadDictionary( 'source_token_dict.pickle')
       # t = loadDictionary(target_token_dict, 'target_token_dict.pickle')
       # t_inv = loadDictionary(target_token_dict_inv, 'target_token_dict_inv.pickle')
        return model, source_token_dict,

In [11]:
x=TestTranslate()
x.test_translate()

i am here: 
AAAA:  (1, 359, 36)
aaaa:  <class 'list'>
XXXX:  359
YYYY:  <class 'int'>
ZZZZ:  359
XXXX2:  359
<class 'int'>
In get_model: encoder_input:  (None, 2589)
In get_model: encoder_embed:  (None, 2589, 32)
Start Warpping................................
OOOO: KerasTensor(type_spec=TensorSpec(shape=(None, 2589, 32), dtype=tf.float32, name=None), name='Encoder-Embedding/add_1:0', description="created by layer 'Encoder-Embedding'")
embed_dim: 32
head_num: 4
dropout_rate: 0.05
masked: False
query_input: XXXXXXX  (None, 2589, 32)
source_input: XXXXXXX  (None, 2589, 32)
Query: XXXXXXX  (None, 2589, 4, 8)
Key: XXXXXXX  (None, 2589, 4, 8)
Value: XXXXXXX  (None, 2589, 4, 8)
dim:  8
query:  Tensor("self_attention_1/query/einsum/Einsum:0", shape=(None, 2589, 4, 8), dtype=float32)
QS OOOOOOOOOOOOO <class 'tensorflow.python.framework.ops.Tensor'> (2589, None, 4, 16)
KS OOOOOOOOOOOOO <class 'tensorflow.python.framework.ops.Tensor'> (2589, None, 4, 16)
Start Warpping............................

Epoch 1/2
query_input: XXXXXXX  (None, 2589, 32)
source_input: XXXXXXX  (None, 2589, 32)
Query: XXXXXXX  (None, 2589, 4, 8)
Key: XXXXXXX  (None, 2589, 4, 8)
Value: XXXXXXX  (None, 2589, 4, 8)
dim:  8
query:  Tensor("model/self_attention_1/query/einsum/Einsum:0", shape=(None, 2589, 4, 8), dtype=float32)
QS OOOOOOOOOOOOO <class 'tensorflow.python.framework.ops.Tensor'> (2589, None, 4, 16)
KS OOOOOOOOOOOOO <class 'tensorflow.python.framework.ops.Tensor'> (2589, None, 4, 16)
query_input: XXXXXXX  (None, 2589, 32)
source_input: XXXXXXX  (None, 2589, 32)
Query: XXXXXXX  (None, 2589, 4, 8)
Key: XXXXXXX  (None, 2589, 4, 8)
Value: XXXXXXX  (None, 2589, 4, 8)
dim:  8
query:  Tensor("model/self_attention_3/query/einsum/Einsum:0", shape=(None, 2589, 4, 8), dtype=float32)
QS OOOOOOOOOOOOO <class 'tensorflow.python.framework.ops.Tensor'> (2589, None, 4, 16)
KS OOOOOOOOOOOOO <class 'tensorflow.python.framework.ops.Tensor'> (2589, None, 4, 16)
query_input: XXXXXXX  (None, 2589, 32)
source_input: XXXXX

InternalError: 2 root error(s) found.
  (0) Internal:  'cuModuleGetFunction(&function, module, kernel_name)' failed with 'CUDA_ERROR_INVALID_HANDLE'
	 [[node model/Encoder-2-FeedForward/mul_5 (defined at D:\Code\Project-AI-JAVA-ANNOTATION-2021\Performer\Perfomer_local\keras_performer\gelu.py:7) ]]
	 [[gradient_tape/model/Token-Embedding/embedding_lookup/Reshape/_108]]
  (1) Internal:  'cuModuleGetFunction(&function, module, kernel_name)' failed with 'CUDA_ERROR_INVALID_HANDLE'
	 [[node model/Encoder-2-FeedForward/mul_5 (defined at D:\Code\Project-AI-JAVA-ANNOTATION-2021\Performer\Perfomer_local\keras_performer\gelu.py:7) ]]
0 successful operations.
0 derived errors ignored. [Op:__inference_train_function_5710]

Function call stack:
train_function -> train_function
